In [1]:
# Movie Recommender System with Neural Networks (Content-Based Filtering)

# 📌 Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# -----------------------------------------
# 1. Create Synthetic Dataset
# -----------------------------------------
# Example users
users = pd.DataFrame({
    "user_id": [1, 2, 3, 4, 5],
    "age": [23, 45, 34, 25, 40],
    "gender": ["M", "F", "M", "F", "M"]
})

# Example movies
movies = pd.DataFrame({
    "movie_id": [101, 102, 103, 104, 105],
    "title": ["Inception", "Titanic", "The Matrix", "Avengers", "Frozen"],
    "genre": ["Sci-Fi", "Romance", "Sci-Fi", "Action", "Animation"]
})

# Ratings (user_id, movie_id, rating)
ratings = pd.DataFrame({
    "user_id": [1, 1, 2, 2, 3, 3, 4, 5],
    "movie_id": [101, 102, 102, 103, 103, 104, 105, 101],
    "rating": [5, 3, 4, 5, 2, 5, 4, 3]
})

print("Users:\n", users)
print("\nMovies:\n", movies)
print("\nRatings:\n", ratings)

# -----------------------------------------
# 2. Preprocessing
# -----------------------------------------
# Encode user_id and movie_id as categorical indices
user_ids = ratings["user_id"].unique().tolist()
movie_ids = ratings["movie_id"].unique().tolist()

user2user_encoded = {x: i for i, x in enumerate(user_ids)}
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}

ratings["user"] = ratings["user_id"].map(user2user_encoded)
ratings["movie"] = ratings["movie_id"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie2movie_encoded)

X = ratings[["user", "movie"]].values
y = ratings["rating"].values

print(f"\nNumber of Users: {num_users}, Number of Movies: {num_movies}")

# -----------------------------------------
# 3. Neural Network Model with Embeddings
# -----------------------------------------
embedding_size = 8

# User embedding
user_input = keras.Input(shape=(1,), name="user")
user_embedding = layers.Embedding(num_users, embedding_size)(user_input)
user_vec = layers.Flatten()(user_embedding)

# Movie embedding
movie_input = keras.Input(shape=(1,), name="movie")
movie_embedding = layers.Embedding(num_movies, embedding_size)(movie_input)
movie_vec = layers.Flatten()(movie_embedding)

# Combine user & movie vectors
concat = layers.Concatenate()([user_vec, movie_vec])
dense = layers.Dense(16, activation="relu")(concat)
output = layers.Dense(1)(dense)

model = keras.Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

model.summary()

# -----------------------------------------
# 4. Training
# -----------------------------------------
history = model.fit(
    x=[X[:, 0], X[:, 1]],
    y=y,
    batch_size=4,
    epochs=30,
    verbose=1
)

# -----------------------------------------
# 5. Recommendation Example
# -----------------------------------------
def recommend_movies(user_id, top_n=3):
    encoded_user = user2user_encoded[user_id]
    movie_arr = np.array(list(movie2movie_encoded.values()))
    user_arr = np.full_like(movie_arr, encoded_user)

    preds = model.predict([user_arr, movie_arr], verbose=0).flatten()
    movie_scores = list(zip(movie_arr, preds))

    top_movies = sorted(movie_scores, key=lambda x: x[1], reverse=True)[:top_n]
    recs = []
    for m_id, score in top_movies:
        movie_real_id = list(movie2movie_encoded.keys())[list(movie2movie_encoded.values()).index(m_id)]
        title = movies[movies["movie_id"] == movie_real_id]["title"].values[0]
        recs.append((title, round(float(score), 2)))

    return recs

# Example: Recommend movies for user 1
print("\nRecommended movies for User 1:")
print(recommend_movies(1))


Users:
    user_id  age gender
0        1   23      M
1        2   45      F
2        3   34      M
3        4   25      F
4        5   40      M

Movies:
    movie_id       title      genre
0       101   Inception     Sci-Fi
1       102     Titanic    Romance
2       103  The Matrix     Sci-Fi
3       104    Avengers     Action
4       105      Frozen  Animation

Ratings:
    user_id  movie_id  rating
0        1       101       5
1        1       102       3
2        2       102       4
3        2       103       5
4        3       103       2
5        3       104       5
6        4       105       4
7        5       101       3

Number of Users: 5, Number of Movies: 5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 8)      │         40 │ user[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 8)      │         40 │ movie[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 8)         │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │        272 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         17 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 369 (1.44 KB)

 Trainable params: 369 (1.44 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 14.8360 - mae: 3.6914 
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 17.3997 - mae: 4.0106
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 15.4911 - mae: 3.8122
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 16.3554 - mae: 3.8906
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 16.6636 - mae: 3.9573
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 14.9009 - mae: 3.7030
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 13.4701 - mae: 3.5215
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 17.7058 - mae: 4.0898
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 16.8722 - mae: 3.9949
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.2155 - mae: 3.7300
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 14.4083 - mae: 3.6358
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 14.3508 - mae: 3.6274
Epoch 13/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/ste